In [1]:
import pymongo
from pprint import pprint
import pandas as pd
import numpy as np
from IPython.display import Image

In [2]:
connection =pymongo.MongoClient("localhost",27017)
db = connection.python
col = db.sales
for i in col.find().limit(1):
    pprint(i)

{'_id': {'id': '5bd761dcae323e45a93ccfe8'},
 'couponUsed': True,
 'customer': {'age': {'$numberInt': '42'},
              'email': 'cauho@witwuta.sv',
              'gender': 'M',
              'satisfaction': {'$numberInt': '4'}},
 'items': [{'name': 'printer paper',
            'price': {'$numberDecimal': '40.01'},
            'quantity': {'$numberInt': '2'},
            'tags': ['office', 'stationary']},
           {'name': 'notepad',
            'price': {'$numberDecimal': '35.29'},
            'quantity': {'$numberInt': '2'},
            'tags': ['office', 'writing', 'school']},
           {'name': 'pens',
            'price': {'$numberDecimal': '56.12'},
            'quantity': {'$numberInt': '5'},
            'tags': ['writing', 'office', 'school', 'stationary']},
           {'name': 'backpack',
            'price': {'$numberDecimal': '77.71'},
            'quantity': {'$numberInt': '2'},
            'tags': ['school', 'travel', 'kids']},
           {'name': 'notepad',
         

### Filter

In [7]:
for i in db.list_collection_names():
    print(i)

posts
sudo
fs.files
chat
sales
login
users
limited
fs.chunks
nei
res


### Creating Test data

In [32]:
import random

def random_gen(name,dic):
    
    for i in range(len(dic['funding_rounds'])):
        dic['funding_rounds'][i]["raised_amount"] = round(random.uniform(500000,10000000))
    dic['name'] = name
    return dic




lst_of_name = ["Facebook","Google","Naver","Daum","Coupang","Baemin","Line"]

for i in lst_of_name:
    dic = {"name":"",
            "funding_rounds":[
            {
            "round_code":"a",
            "raised_amount":0},
             {
            "round_code":"b",
            "raised_amount":0},
             {
            "round_code":"c",
            "raised_amount":0},
             {
            "round_code":"d",
            "raised_amount":0},
             {
            "round_code":"e",
            "raised_amount":0},
             {
            "round_code":"f",
            "raised_amount":0}
            ]}
    db.funds.insert_one(random_gen(i,dic))
#빈 딕셔너리를 안에 지정하지 않으면 같은 dictionary로 인식되어 중복 키 에러를 일으킬 것

In [35]:
for i in db.funds.find():
    pprint(i)

{'_id': ObjectId('61167af198608fe75d51477e'),
 'funding_rounds': [{'raised_amount': 7172702, 'round_code': 'a'},
                    {'raised_amount': 7914665, 'round_code': 'b'},
                    {'raised_amount': 7915102, 'round_code': 'c'},
                    {'raised_amount': 9297130, 'round_code': 'd'},
                    {'raised_amount': 7519725, 'round_code': 'e'},
                    {'raised_amount': 3479603, 'round_code': 'f'}],
 'name': 'Facebook'}
{'_id': ObjectId('61167af198608fe75d51477f'),
 'funding_rounds': [{'raised_amount': 2716902, 'round_code': 'a'},
                    {'raised_amount': 9737827, 'round_code': 'b'},
                    {'raised_amount': 9606464, 'round_code': 'c'},
                    {'raised_amount': 618771, 'round_code': 'd'},
                    {'raised_amount': 509318, 'round_code': 'e'},
                    {'raised_amount': 1924125, 'round_code': 'f'}],
 'name': 'Google'}
{'_id': ObjectId('61167af198608fe75d514780'),
 'funding_rounds':

### Filter적용
- \$match
- \$expr
- \$strLenCP
- \$filter
- \$gt   
    - aggregation에서 gt는 배열형태 띄고 첫번째를 비교할 인자, 두번째는 기준 숫자가 된다.
    - "\\$gt": \[\{"\\$strLenCP": "\\$name" }, 6 ]  

In [41]:
#이름이 몇 글자 이상인 것만 필터링 하겠다.
for i in db.funds.aggregate([{"$match":{"$expr": { "$gt": [ { "$strLenCP": "$name" }, 6 ] }}}]):
    pprint(i)

{'_id': ObjectId('61167af198608fe75d51477e'),
 'funding_rounds': [{'raised_amount': 7172702, 'round_code': 'a'},
                    {'raised_amount': 7914665, 'round_code': 'b'},
                    {'raised_amount': 7915102, 'round_code': 'c'},
                    {'raised_amount': 9297130, 'round_code': 'd'},
                    {'raised_amount': 7519725, 'round_code': 'e'},
                    {'raised_amount': 3479603, 'round_code': 'f'}],
 'name': 'Facebook'}
{'_id': ObjectId('61167af198608fe75d514782'),
 'funding_rounds': [{'raised_amount': 8544121, 'round_code': 'a'},
                    {'raised_amount': 2108116, 'round_code': 'b'},
                    {'raised_amount': 3454778, 'round_code': 'c'},
                    {'raised_amount': 4738163, 'round_code': 'd'},
                    {'raised_amount': 2407252, 'round_code': 'e'},
                    {'raised_amount': 573930, 'round_code': 'f'}],
 'name': 'Coupang'}


#### \\$filter
- 여러가지 조건을 가진다.<br>
    1) 입력될 배열 - input : 
    2) 입력될 배열을 참조하기 위한 이름 - as : 
    3) 조건 - cond : 

In [51]:
for i in db.funds.aggregate([{"$match":{"$expr": { "$gt": [ { "$strLenCP": "$name" }, 6 ] }}},
                             {"$project":{
                                 "_id":0,
                                 "name":1,
                                 "amount":{"$filter":{
                                     "input":"$funding_rounds", #어떤 배열이 인풋 정보가 될 것인가.
                                     "as":"round",
                                     "cond":{"$gte":["$$round.raised_amount",4738163]} #해당 금액 이상의 배열만 출력한다.
                                 }}
                             }}]):
    pprint(i)
    
    

{'amount': [{'raised_amount': 7172702, 'round_code': 'a'},
            {'raised_amount': 7914665, 'round_code': 'b'},
            {'raised_amount': 7915102, 'round_code': 'c'},
            {'raised_amount': 9297130, 'round_code': 'd'},
            {'raised_amount': 7519725, 'round_code': 'e'}],
 'name': 'Facebook'}
{'amount': [{'raised_amount': 8544121, 'round_code': 'a'},
            {'raised_amount': 4738163, 'round_code': 'd'}],
 'name': 'Coupang'}


#### \\$arrayElemAt


In [65]:
for i in db.funds.aggregate([{"$match":{"$expr": { "$gt": [ { "$strLenCP": "$name" }, 6 ] }}},
                             {"$project":{
                                 "_id":0,
                                 "name":1,
                                 "first_round":{"$arrayElemAt":["$funding_rounds.raised_amount",0]},
                                 "final_round":{"$arrayElemAt":["$funding_rounds.raised_amount",-1]},
                                 }
                             },
                            {"$project":{
                                 "_id":0,
                                 "name":1,
                                 "first_round":1,
                                 "final_round":1,
                                 "difference":{"$subtract":["$first_round","$final_round"]}}}]):
    pprint(i)
#why did I use project twice? it's because it cannot refer to things that hasn't assigned the value to.

{'difference': 3693099,
 'final_round': 3479603,
 'first_round': 7172702,
 'name': 'Facebook'}
{'difference': 7970191,
 'final_round': 573930,
 'first_round': 8544121,
 'name': 'Coupang'}
